Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

***Модели будем обучать с помощью алгоритмов RandomForestClassifier, LogisticRegression, CatBoostClassifier*** 

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

***Подгрузим библиотку nltk для подготовки текста***

In [5]:
import nltk

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import WhitespaceTokenizer
lemmatizer = WordNetLemmatizer() 
w_tokenizer = WhitespaceTokenizer()

***Подгрузим буржуйские стопвордс***

In [11]:
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('english'))

In [12]:
import warnings
warnings.filterwarnings(action='once')

In [13]:
pwd

'/home/user-1-1120000000218418/work'

data = pd.read_csv('/datasets/toxic_comments.csv')

In [14]:
data = pd.read_csv('/datasets/toxic_comments.csv')

***Приступим к очистке данных***

***Пишем функцию, которая будет чистить от лишних символов***

In [1]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(
        lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", elem))   
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", " ", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x.split()))
    return df

In [16]:
clean_text(data, 'text', 'text_cleaned')

,text,toxic,text_cleaned
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...
159569,And it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...


***Проведем токенизацию и лемантизацию***

In [17]:
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i  in w_tokenizer.tokenize(text)]
    lem_text = " ".join(lem_text)
    return lem_text

In [18]:
data['text_cleaned_lemmatized'] = data['text_cleaned'].apply(lambda x: word_lemmatizer(x))
data.head()

,text,toxic,text_cleaned,text_cleaned_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,you sir are my hero any chance you remember wh...


***Делим данные на тренировочную и тестовые выборки в отношении 1 к 1***

In [19]:
train, test =  train_test_split(data, shuffle=True, test_size=0.5)

***Создадим корпус постов для тренировки и теста***

In [20]:
corpus = train['text_cleaned_lemmatized'].values.astype('U')

In [21]:
corpus_test = test['text_cleaned_lemmatized'].values.astype('U')

***Создадим счётчик, указав в нём стоп-слова. Посчитаем TF-IDF для корпуса тренировочных текстов, вызовем функцию fit_transform()***

In [22]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

In [23]:
tf_idf.shape

(79785, 102291)

***Посчитаем TF-IDF для корпуса тестовы текстов, вызовем функцию transform()***

In [24]:
tf_idf_test = count_tf_idf.transform(corpus_test)

In [25]:
tf_idf_test.shape

(79786, 102291)

# 2. Обучение

***Метрикой измерения успеха обучения выберем f1, при достижении отметки 0.75***

In [26]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')

***Напишем функию для выбранных алгоритмов, воспользуем рандомным поиском, для подборки оптимальных гиперпараметров***

In [27]:
def f1_model_random(model):
    f1 = 0
    best = 0
    grid = 0
    
    #________________________________________________________________________________________
    if model in "LogisticRegression":
        grid = {
        "C":[1.0,5.0, 10.0], 
        #"class_weight":[True, False],
        #"dual":[False],
        #"fit_intercept":[True, False],
        #"intercept_scaling":[1],
        #"l1_ratio":[None], 
        'max_iter':[int(x) for x in np.linspace(start = 1000, stop = 5500, num = 5)],
        #'multi_class':['auto'],
        #'n_jobs':[None],
        #'penalty':['l2'],
        'random_state':[12345], 
        #'solver':['lbfgs','sag'], 
        #'tol':[0.0001], 
        'verbose':[0],     
        #'warm_start':[True, False]  
    }
    
        logreg=LogisticRegression();
        logreg_cv=RandomizedSearchCV(estimator = logreg,param_distributions = grid,scoring = 'f1',
                                     n_iter = 5,cv =3, verbose=0, random_state=12345, n_jobs=-1);
        logreg_cv.fit(tf_idf, train['toxic']);
        best = logreg_cv.best_params_        
    
        logreg=LogisticRegression(**best)
        logreg.fit(tf_idf, train['toxic'])
        predicted_test = logreg.predict(tf_idf_test)
        f1=f1_score(test.toxic, predicted_test)
        print("LogisticRegression f1_test:",f1)
 #__________________________________________   
    if model in "RandomForestClassifier":
        grid = {
            'n_estimators':[int(x) for x in np.linspace(start = 10, stop = 110, num = 5)],
            #'criterion':['gini'],
            #'max_depth':[None],
            'min_samples_split':[2, 4],
            'min_samples_leaf':[1, 2],
            #'min_weight_fraction_leaf':[0.0],
            #'max_features':['auto'],
            #'max_leaf_nodes':[None],
            #'min_impurity_decrease':[0.0],
            #'min_impurity_split':[None],
            #'bootstrap':[True],
            #'oob_score':[False],
            #'n_jobs':[None],
            'random_state':[None],
            'verbose':[0],
            #'warm_start':[False],
            #'class_weight':[None],
            #'ccp_alpha':[0.0],
            #'max_samples':[None],
        }
    
        ranforclas=RandomForestClassifier();
        ranforclas_cv=RandomizedSearchCV(estimator = ranforclas,param_distributions = grid,scoring = 'f1',
                                     n_iter = 5,cv =3, verbose=0, random_state=12345, n_jobs=-1);
        ranforclas_cv.fit(tf_idf, train['toxic']);
        best = ranforclas_cv.best_params_        
    
        ranforclas=RandomForestClassifier(**best)
        ranforclas.fit(tf_idf, train['toxic'])
        predicted_test = ranforclas.predict(tf_idf_test)
        f1=f1_score(test.toxic, predicted_test)
        print("RandomForestClassifier f1_test:",f1)
 #__________________________________________   
    if model in "CatBoostClassifier":
        grid = {
            #'early_stopping_rounds':[10],
            'iterations':[int(x) for x in np.linspace(start = 50, stop = 100, num = 2)],
            'learning_rate':[float(x) for x in np.linspace(start = 0.1, stop = 1, num =2)],
            'depth':[int(x) for x in np.linspace(start = 2, stop = 16, num = 2)]}
        
        cbc= CatBoostClassifier() # классификатор
        cbc_cv=RandomizedSearchCV(estimator = cbc,param_distributions = grid,scoring = 'f1',
                                  n_iter = 1,cv =3, verbose=0, random_state=12345);
        cbc_cv.fit(tf_idf, train['toxic']); # обучение классификатора
        best = cbc_cv.best_params_  
        
        cbc=CatBoostClassifier(**best)
        cbc.fit(tf_idf, train['toxic'])
        predicted_test = cbc.predict(tf_idf_test)
        f1=f1_score(test.toxic, predicted_test)
        print("CatBoostClassifier f1_test:",f1)

***Обучим модель алгоритмом CatBoostClassifier***

***Обучим модель алгоритмом LogisticRegression***

In [ ]:
f1_model_random('LogisticRegression')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


***Обучим модель алгоритмом RandomForestClassifier***

# 3. Выводы

1. Были загружены бибилиотки для обучения и подготовки данных
2. Проведена лемантизация, токенезация и чистка текстов
3. Созданы корпуса на обучающей и тесвовой выборках
4. Посчитаны TF-IDF
5. Обучены три алгоритма с поиском лучших гиперпараметров на рандомном поиске и выбрана лучшая модель LogisticRegression с наибольшим показателем F1 = 0.764

# 3.3 Выводы выводов

1. Меньшей проблемой было очистить данные и подготовить корпуса.
2. Самая частая проблема мертвый kernel при расчете модели, только когда считал на локальной машине это удалось преодолеть. Время расчетов слишком большое, попытки прикрутить расчет через GPU для CatBoost не увенчались успехом.
3. Об использовании BERT даже не приходится мечтать.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны